## Final Project Submission

Please fill out:
* Student name: Deztany Jackson
* Student pace: self paced 
* Scheduled project review date/time: June 22, 2022
* Instructor name: Claude Fried
* Blog post URL: http://dmvinedata.com/blog/



## Microsoft Movie Studio Analysis

#### Authors: Deztany Jackson

## Overview 

This analysis directs Microsoft's potential studio head with actional insights on their path forward into developing a movie studio. These recommendations are determined from insights from box office movie "Ratings" and depends on other attributes such as "Genre", "Directors" and "Movie Budget". The datasets given were filtered to analyse movies and their associated  with a minimum of a  "8/10" ("B") rating. The analysis goal was to extract patterns from some of the "best" movies.

#RESULTS
#RECOMMENDATIONS




## Business Problem

Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

Microsft is in competition with current major studios which have been in business for many years and already have a wealth of experience. The questions driving the analysis, support  Microsoft making choices based on direct data output from their  competition as well as  potential growth areas that may be overlooked. The analysis questions and main variable of success were chosen with the mindset, Microsoft is a major tech company that has influence and skillsets in other areas of tech. 

#They have areas other usecases that can exploit findings"Ratings" are the mainThe potential growth areas are asked with the context of knowing Microsoft's technical coverage that can exploi a  a wealth of experienc

Ref: [Phase 1 Project Description](https://learning.flatironschool.com/courses/4963/pages/phase-1-project-description?module_item_id=370765), 2022

## Data Understanding

Box office related datasets (SQL and CSV) with the box office movie target "Rating" variable and associated independent  "Genre, Director and Budget" variable information  were taken from [IMBD](https://www.imdb.com/) and [the-numbers](https://www.the-numbers.com/). 

In [1]:
#Import libraries

import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
#Allows querying of dataframe with SQL
import pandasql as sqldf


In [3]:
#Lambda function to query datafram with SQL
pysqldf = lambda q: sqldf(q, globals())

In [5]:
#Connecting with databases
conn = sqlite3.connect('im.db')


In [9]:
#View the list of tables in the "IMDB" database

df = pd.read_sql("""
SELECT name
FROM sqlite_master 
WHERE type = 'table';""", conn)
df

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers



### IMBD Tables Used:

#### - movie_basics
    - The movie_basics dataset describes just over      146,000 movies with their associated genres, years.
#### - movie_ratings
    - The movie_ratings dataset describes 73,856 movie's       average rating (from average ratings that range         from "1" to "10") and the number of votes the           average is calculated (ranging from "5" to "1.84         Mil"). 
#### - directors
     - The directors dataset describes 291,174
       movies and their associated director's id.
 
#### - persons
    - The persons dataset describes 606,648 people and       their associated movie related profession(s). 

#### IMBD Table: movie_basics

In [60]:
#Looking at movie_basic table
q_basics = '''
SELECT
    *
FROM movie_basics

;
'''

MovBasics_df = pd.read_sql(q_basics, conn)
#What does the dataset include?
MovBasics_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [53]:
MovBasics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [54]:
MovBasics_df["genres"].value_counts()

Documentary                   32185
Drama                         21486
Comedy                         9177
Horror                         4372
Comedy,Drama                   3519
                              ...  
Adventure,Music,Mystery           1
Documentary,Horror,Romance        1
Sport,Thriller                    1
Comedy,Sport,Western              1
Adventure,History,War             1
Name: genres, Length: 1085, dtype: int64

In [48]:
MovBasics_df.isna().sum()

movie_id               0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

In [72]:
#Looking to see if there are any duplicates
q_basics = '''
SELECT
    movie_id,
    start_year,
    primary_title,
    genres,
    COUNT(*) AS CNT
FROM movie_basics
GROUP BY movie_id
HAVING COUNT(*) > 1
ORDER BY movie_id
;
'''

MovBasics_df = pd.read_sql(q_basics, conn)
#Displays if there are duplicate movies
MovBasics_df

,movie_id,start_year,primary_title,genres,CNT


#### IMBD Table: movie_ratings

In [31]:
#Looking at movie_ratings table
q_basics = '''
SELECT
    *
FROM movie_ratings
;
'''

MovRatings_df = pd.read_sql(q_basics, conn)
MovRatings_df.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [30]:
MovRatings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [121]:
MovRatings_df.describe()

,averagerating,numvotes
count,73856.000000,7.385600e+04
mean,6.332729,3.523662e+03
std,1.474978,3.029402e+04
min,1.000000,5.000000e+00
25%,5.500000,1.400000e+01
50%,6.500000,4.900000e+01
75%,7.400000,2.820000e+02
max,10.000000,1.841066e+06


In [47]:
MovRatings_df.isna().sum()

movie_id         0
averagerating    0
numvotes         0
dtype: int64

#### IMBD Table: directors

In [50]:
#Looking at directors table
q_basics = '''
SELECT
    *
FROM directors
;
'''

MovDirID_df = pd.read_sql(q_basics, conn)
MovDirID_df.head()

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [55]:
MovDirID_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [56]:
MovDirID_df.describe()

,movie_id,person_id
count,291174,291174
unique,140417,109253
top,tt4050462,nm6935209
freq,3818,238


In [51]:
MovDirID_df.isna().sum()

movie_id     0
person_id    0
dtype: int64

#### IMBD Table: persons

In [43]:
#Looking at persons table
q_basics = '''
SELECT
    *
FROM persons
;
'''

MovDir_df = pd.read_sql(q_basics, conn)
MovDir_df.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [61]:
MovDir_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
dtypes: float64(2), object(3)
memory usage: 23.1+ MB


In [46]:
MovDir_df["primary_name"].isna().sum()

0

#### Budget CSV: movie_budgets
    - This dataset describes the budget and gross from 5782 movies. The movie name values are the only link to joining the these values with other movie datasets.

In [70]:
#Import CSV to a dataframe
Bud_df = pd.read_csv('movie_budgets.csv')
Bud_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [69]:
#Note that the production_budget is a string and not a number
Bud_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


## Data Preparation
Describe and justify the process for preparing the data for analysis.

### Data Cleaning

#### movie_basic, movie_ratings, directors, persons

###  Data Merge and Clean:                                        
movie_basic and movie_ratings 

In [88]:
#Joining movie_basics and ratings
qB1 = """
SELECT
    movie_id,
    start_year,
    primary_title,
    genres,
    averagerating,
    numvotes
FROM movie_basics
JOIN movie_ratings
    USING (movie_id)
GROUP BY movie_id
ORDER BY numvotes DESC, averagerating
;
"""
Genre_Ratings= pd.read_sql(qB1,conn)

In [89]:
movieG_df = pd.DataFrame(Genre_Ratings)
movieG_df.head()

,movie_id,start_year,primary_title,genres,averagerating,numvotes
0,tt1375666,2010,Inception,"Action,Adventure,Sci-Fi",8.8,1841066
1,tt1345836,2012,The Dark Knight Rises,"Action,Thriller",8.4,1387769
2,tt0816692,2014,Interstellar,"Adventure,Drama,Sci-Fi",8.6,1299334
3,tt1853728,2012,Django Unchained,"Drama,Western",8.4,1211405
4,tt0848228,2012,The Avengers,"Action,Adventure,Sci-Fi",8.1,1183655


[Ref: Split Values on Rows on Stackoverflow ](https://stackoverflow.com/questions/17116814/pandas-how-do-i-split-text-in-a-column-into-multiple-rows)

In [90]:
#Spliting the genres into sep rows by ","
s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1) # align index
s.name = "genres" #name to join on
s

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

...

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859/2146536929.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = movieG_df["genres"].str.split(",").apply(pd.Series, 1).stack()
/var/folders/77/kvw5t075169_rvys7qfdc_xw0000gn/T/ipykernel_84859

0             Action
0          Adventure
0             Sci-Fi
1             Action
1           Thriller
            ...     
73851          Drama
73852    Documentary
73853    Documentary
73854    Documentary
73855    Documentary
Name: genres, Length: 128490, dtype: object

In [91]:
#delete old dataframe column with multiple values
del movieG_df["genres"]



In [92]:
#making it equal to the new dataframe
movieG1_df = movieG_df.join(s)
movieG1_df.head()

#### Missing Values
Drop missing values from dataframe.

In [93]:
#drop missing values 
movieG1_df = movieG1_df.dropna()

In [119]:
movieG1_df["genres"].unique().sum()

'ActionAdventureSci-FiThrillerDramaWesternBiographyCrimeMysteryComedyFantasyFamilyAnimationRomanceMusicHistoryHorrorSportWarMusicalDocumentaryNewsGame-ShowReality-TVAdultShort'

In [120]:
# describing the new table with movie genres(each row ) and ratings
movieG1_df.describe()



,start_year,averagerating,numvotes
count,128490.000000,128490.000000,1.284900e+05
mean,2014.221021,6.302146,5.337769e+03
std,2.579176,1.457744,3.808942e+04
min,2010.000000,1.000000,5.000000e+00
25%,2012.000000,5.400000,1.600000e+01
50%,2014.000000,6.400000,6.600000e+01
75%,2016.000000,7.300000,4.290000e+02
max,2019.000000,10.000000,1.841066e+06


In [ ]:
#Checking datatypes..helps to see how I can use the ratings
ax = g.plot.scatter("genres", "numvotes" , c = "averagerating" , s = g["averagerating"], colormap = "hsv",xlabel = "genres", ylim = [50000,100000],  figsize = (20,20))
ax.set_xlabel("Genres")
ax.set_title("Title Example")


In [ ]:
#Measures of c
movieG1_df["numvotes"].describe()

In [ ]:
#Checking to see if any ratings values have missing data
movieG_df["averagerating"].isna().sum()

In [ ]:
#There are 804 places where te data is missing...We can fill in the missing genre with "Unknown".  Allow us to drop in the
#future or further examine and add from other datasets without losing it
movieG_df["genres"].isna().sum()

In [ ]:
#Possibly separate our genres seperately...write a function to separate 
movieG_df["genres"]

In [ ]:
movieG1_df.describe()

In [ ]:
# Rename values in genre to match
movieG1_df["genres"].replace("Musical","Music", inplace = True)
                      

In [ ]:
#The limits of the dataset to plot for rating.  #Mean ~5337Change hard coded numbers to average
g = movieG1_df.loc[(movieG1_df["averagerating"] >=8) & (movieG1_df["numvotes"] > movieG1_df["numvotes"].mean())]


In [ ]:
#Understanding the number of directors per genre with preconditions
g.sort_value(["movie_id"]).groupby("genres").count()

In [ ]:
g.describe()

In [ ]:
fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize=(30, 30))
g.plot.scatter(x = "genres", y = "numvotes", c = "averagerating" , s = 100,  colormap = "rainbow",   ax=ax)
ax.set_title('Distribution of averagerating for genres')

ax.set_yscale('log')


### Movie Direct

In [ ]:
#Why is there more rows when I group the person_id after the movie id...There could be diff directors for the same movie
#Joining movie_b and ratings and directors
qD = """
SELECT
    movie_id,
    averagerating,
    numvotes,
    person_id,
    primary_name AS Director
    
FROM movie_basics
JOIN movie_ratings
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING(person_id)
GROUP BY movie_id, person_id
ORDER BY movie_id

;
"""
Dir_Sql = pd.read_sql(qD,conn)

In [ ]:
#tHIS IS THE rating and directors joined...above 8 with minimum amount of votes

q = """
SELECT 
    g.movie_id,
    g.primary_title,
    g.genres,
    g.averagerating,
    g.numvotes,
    Director,
    COUNT (genres) AS CNTg,
    COUNT (DISTINCT movie_id) AS CNTm,
    COUNT (DISTINCT genres) AS CNTg,
    ROUND (AVG(g.averagerating), 1) as AvgRat_Dir
FROM g
JOIN movieD_df
    USING (movie_id)
GROUP BY director
ORDER BY AvgRat_Dir DESC, g.numvotes DESC, g.genres DESC
"""

intDir = pysqldf(q)
intDir

In [ ]:
#tHIS IS THE rating and directors joined...above 8 with minimum amount of votes

q = """
SELECT 
    g.movie_id,
    g.primary_title,
    g.genres,
    g.averagerating,
    g.numvotes,
    Director

FROM g
JOIN movieD_df
    USING (movie_id)

ORDER BY  g.numvotes DESC, g.genres DESC;
"""

intDir = pysqldf(q)
intDir

In [ ]:
movieD_df = pd.DataFrame(Dir_Sql)
movieD_df

In [ ]:
fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize=(30, 30))
intDir.plot(x = "Director", y = "genres", c = "orange" , s = 100,  colormap = "rainbow",   ax=ax)
ax.set_title('Distribution of averagerating for genres')



In [ ]:
#How many movies within the  top Genres does each director have?


In [ ]:
#How many movies does each director have (with preconditions) 
#How many different type of genres does each director have (with preconditions)
#Which genres does each director have
#COUNT (genres) AS CNTg,
   # COUNT ( DISTINCT movie_id) AS CNTm,
   # COUNT (DISTINCT genres) AS DCNTg
# You can put dataframe with group
q = """
SELECT 
    director,
    genres,
    COUNT ( DISTINCT movie_id) AS Movie_Count,
    COUNT (DISTINCT  genres) AS Genre_Count
FROM g
JOIN movieD_df
    USING (movie_id)
WHERE genres IN ("Drama", "Action", "Crime", "Comedy", "Thriller")    
GROUP BY director
ORDER BY Movie_Count DESC
;

"""

intDir = pysqldf(q)
intDir

In [ ]:
#Count the number of movies each genre has
#tHERE ARE 21 Genres

mC ="""

SELECT
    movie_id,
    start_year,
    COUNT (movie_id) AS MCNT,
    genres,
    primary_title,
    averagerating,
    numvotes

FROM g
GROUP BY genres
ORDER BY  MCNT DESC
;


"""

MovieCount = pysqldf(mC)
MovieCount

In [ ]:
#Median number of votes per genre...with preconditions...SQL is hard to do median...skewed set
#Look at genre with top ratings and middle number of movies - INSIGHT into potential areas...because of avg and number of votes
#Change title or give stats for them##
gMC = MovieCount.groupby(["genres"]).median()
gMC.sort_values(["MCNT","numvotes", "averagerating" ], ascending = False)

In [ ]:
#Keep Table COUNT (movie_id) AS MCNT,
#Number of Directors in each genre with  preconditions
q = """
SELECT 
   
   movie_id,
   genres,
   COUNT (Director) AS DirCount
FROM g
JOIN movieD2_df
    USING (movie_id)
GROUP BY genres
ORDER BY DirCount DESC

;

"""

GenDir = pysqldf(q)
GenDir

In [ ]:
#GOOD TABLE
#Which genres does each director have
#COUNT (genres) AS CNTg,
   # COUNT ( DISTINCT movie_id) AS CNTm,
   # COUNT (DISTINCT genres) AS DCNTg
# This takes into account all the directors list added
q = """
SELECT 
    director,
    genres,
    COUNT ( DISTINCT movie_id) AS Movie_Count,
    COUNT (DISTINCT  genres) AS Genre_Count
FROM g
JOIN movieD2_df
    USING (movie_id)
GROUP BY director
HAVING Movie_Count > 1
ORDER BY Movie_Count DESC
;

"""

Dir = pysqldf(q)
Dir

In [ ]:
#Table for plotting Top 25 Directs more than 2 movies and genre by ratings
#Which genres does each director have
#COUNT (genres) AS CNTg,
   # COUNT ( DISTINCT movie_id) AS CNTm,
   # COUNT (DISTINCT genres) AS DCNTg
# This takes into account all the directors list added
 #ROUND (AVG(numvotes), 1) as Avg_Votes
    #ROUND (AVG(averagerating), 1) as Avg_Rat,
q = """
SELECT 
    director,
    movie_id,
    numvotes,
    ROUND (AVG(numvotes), 1) as Avg_Votes,
    averagerating,
    ROUND (AVG(averagerating), 1) as Avg_Rat,
    genres
FROM g
JOIN movieD2_df
    USING (movie_id)
GROUP BY  director, movie_id, genres
ORDER BY averagerating DESC
;

"""

Dir_df = pysqldf(q)
Dir_df

# Director Visual

In [ ]:
Dir_df

In [ ]:
fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize=(50, 30))
Dir_df.plot.scatter(x = "Director", y = "genres", c = "averagerating" ,s = 150,  colormap = "rainbow",   ax=ax)
ax.set_title('Distribution of Genres by Budget')

In [ ]:
#Do a pivot table to show genres grouped
Dir.groupby(["genres"]).count()

In [ ]:
#Average rating of movies for the director in (with votingnum  preconditions only)
#>2 Movies re only 24 directors, with 1 we can possibly give chance to...They have credibility to their name


qD = """
SELECT
    movie_id,
    genres,
    numvotes,
    person_id,
    primary_title,
    primary_name AS Director,
    COUNT ( DISTINCT movie_id) AS Movie_Count,
    COUNT (DISTINCT  genres) AS Genre_Count,
    ROUND (AVG(averagerating), 1) as AvgRat
FROM movie_basics
JOIN movie_ratings
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING(person_id)
WHERE numvotes >= "5330"
GROUP BY Director
HAVING AvgRat >= 8
ORDER BY AvgRat DESC, Movie_Count DESC

;
"""
Dir_Sql = pd.read_sql(qD,conn)

movieD1_df = pd.DataFrame(Dir_Sql)
movieD1_df

In [ ]:
#Keep Table
#Number of Directors in each genre with  preconditions
gD_count = GenDir.groupby("genres").count().sort_values("Director", ascending = False)
gD_count

In [ ]:
#Join director and broken out genre table, only list the top 5 genres from these directors
#How many directors fit under the top 5 genres each - 13
#From the directors which ones have the top 5 genres
#WHERE g.genres in ("Drama","Action", "Comedy", "Thriller", "Documentary")
#Count the number of movies in each genre with preconditions
qdr ="""

SELECT
   g.movie_id,
   g.genres,
   g.primary_title,
   g.averagerating,
   Director,
   Movie_Count,
   Genre_Count,
   AvgRat
FROM g
INNER JOIN movieD1_df AS  mD
    USING (movie_id)

GROUP BY movie_id


;

"""


int1Dir = pysqldf(qdr)
int1Dir

In [ ]:
#Joining of the the tables


qD = """
SELECT
    movie_id,
    primary_name AS Director
    
FROM movie_basics
JOIN movie_ratings
    USING (movie_id)
JOIN directors
    USING (movie_id)
JOIN persons
    USING(person_id)



;
"""
Dir_Sql = pd.read_sql(qD,conn)

movieD2_df = pd.DataFrame(Dir_Sql)
movieD2_df

In [ ]:
#How many different type of genres does each director have 

In [ ]:
#Top 25 directors Scatter rankings

In [ ]:
#directors only
sql_query = pd.read_sql_query ('''
                               SELECT
                             *
                               FROM directors
                               JOIN persons
                                   USING (person_id)
                            
                               ORDER BY person_id
                               ''', conn)

movieD_df = pd.DataFrame(sql_query)
movieD_df

# Look at Budget- state limitations
# Use

In [ ]:
#1544 Rows
q = """
SELECT 
   averagerating,
   numvotes,
   movie_id,
   genres,
   Director,
   primary_title
FROM g
JOIN movieD2_df
    USING (movie_id)



;

"""

BudMovie = pysqldf(q)
BudMovie

In [ ]:
#Remove necessary characters from string
Bud_df["production_budget"]=Bud_df["production_budget"].str.strip("&")
Bud_df["production_budget"]=Bud_df["production_budget"].str.replace(",", "")

In [ ]:

#Convert Budget from string to int necessary
Bud_df["production_budget"]=Bud_df["production_budget"].astype(int) 
Bud_df.describe()

In [ ]:
#Joining by names narrowed the list to 638 rows
q = """
SELECT
*
FROM BudMovie AS bm
JOIN Bud_df bd
    ON bm.primary_title = bd.movie
;

"""
bud_join= pysqldf(q)
bud_join.info()

In [ ]:
#Refine the joined budget list.  
#Look to see if the top genres are top and if the same directors for the same movies are still good
q ="""
SELECT
    movie_id,
    movie,
    Director,
    genres,
    averagerating,
    numvotes,
    production_budget AS pbud
FROM bud_join
ORDER BY pbud DESC
;

"""
budget_df= pysqldf(q)
budget_df

In [ ]:
#Formatting Budget https://www.markhneedham.com/blog/2021/04/11/pandas-format-dataframe-numbers-commas-decimals/
#budget_df = budget_df.style.format({
    "pbud": "{:,d}",})


In [ ]:
budget_df.info()




In [ ]:
#Look at average budget for genres

q ="""
SELECT
    movie_id,
    movie,
    Director,
    genres,
    averagerating,
    ROUND (AVG(numvotes), 0) as Avg_Votes,
    pbud,
    ROUND (AVG(pbud), 0) as Avg_Bud
FROM budget_df
GROUP BY genres
ORDER BY Avg_Bud DESC, Avg_Votes DESC
;
"""
budA_df= pysqldf(q)
budA_df

In [ ]:
#Look at top directors (5) Are they on the list

#Drama 86, Biogrpahy 26, Crime 15, Doc and Mystery 10 for representations with under 100 Mill budget
q ="""
SELECT
    movie_id,
    movie,
    Director,
    genres,
    averagerating,
    ROUND (AVG(numvotes), 0) as Avg_Votes,
    pbud,
    COUNT (genres) AS Gen_Count,
    ROUND (AVG(pbud), 0) as Avg_Bud
FROM budget_df
GROUP BY director, genres
HAVING Avg_Bud < 100000000
ORDER BY Gen_Count DESC, Avg_Bud ASC,Avg_Votes DESC 
;
"""
budDir_df= pysqldf(q)
budDir_df

In [ ]:
#Plot the genres by the budget


q ="""
SELECT
    movie_id,
    genres,
    averagerating,
    pbud
FROM budget_df
ORDER BY pbud
;
"""
budGen_df= pysqldf(q)
budGen_df

In [ ]:
#Plot the genres by Budget
fig, ax = plt.subplots(ncols = 1, nrows = 1, figsize=(30, 30))
budGen_df.plot.scatter(x = "genres", y = "pbud" , c = "averagerating" ,s = 150,  colormap = "rainbow",   ax=ax)
ax.set_title('Distribution of Genres by Budget')